# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [3]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [5]:
prompt = f"""
I need a report on the Wimbledon tennis tournament and need to discuss the latest match. 
Find out who the 2023 winner was for the mens and womens singles.
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: As an AI language model, I cannot provide information about future events as they have not yet occurred. The Wimbledon tennis tournament for 2023 has not taken place yet, and therefore, there is no information available about the winners of the men's and women's singles. However, I can provide information about the previous Wimbledon tournament.

The 2021 Wimbledon tournament was held from June 28 to July 11, 2021, at the All England Lawn Tennis and Croquet Club in London, England. The tournament was canceled in 2020 due to the COVID-19 pandemic.

In the men's singles, Novak Djokovic of Serbia won his sixth Wimbledon title by defeating Matteo Berrettini of Italy in the final. Djokovic won the match in four sets, 6-7(4), 6-4, 6-4, 6-3. With this victory, Djokovic tied Roger Federer and Rafael Nadal for the most Grand



¿Es esta la respuesta que esperabas?

No, puesto que al modelo le hace falta información para que pueda sacar una conclusión.

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [6]:
text="""
El torneo de Wimbledon de 2023 fue un campeonato de tenis que se disputó entre el 3 al 16 de julio del 2023, en las pistas de hierba del All England Lawn Tennis and Croquet Club, ubicado en Wimbledon (Reino Unido). Fue la 136.ª edición del campeonato y el tercer torneo de Grand Slam del año, cuyas preliminares se jugaron entre el 26 al 29 de junio del presente año.1​2​

Fue el primer torneo que se llevó a cabo bajo el reinado de Carlos III del Reino Unido.

Además contó con la vuelta de los campeones defensores en individuales masculinos y femeninos, Novak Djokovic y Elena Rybakina, respectivamente.

Este torneo fue el último en la carrera profesional de Anett Kontaveit, quien se retiró prematuramente del tenis a los 27 años debido a una hernia discal lumbar en su espalda. La ex número 2 del mundo disputó su último partido en la segunda ronda ante Marie Bouzková.

Carlos Alcaraz en individuales masculinos y Markéta Vondroušová, en individuales femeninos respectivamente, se proclamaron campeones.
"""

prompt = f"""
Quiero saber quienes fueron los ganadores del torneo Wimbledon 2023 en las categorias individuales femenino y marculino.

Context: '''{text}'''
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: Los ganadores del torneo Wimbledon 2023 en las categorías individuales femenino y masculino fueron Markéta Vondroušová y Carlos Alcaraz, respectivamente.



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe? Si, este contexto ayudó al modelo a dar una mejor respuesta

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
